In [25]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd  # Example, if using tabular data
import numpy as np
import cv2 as cv
from PIL import Image
import imageio
import matplotlib.pyplot as plt
import sys
import os
import import_ipynb
from tqdm import tqdm
#import image_utils as iu
%run ./image_utils.ipynb
%run ./clustering.ipynb
BaseFolder = '../../datasets/'
positions = BaseFolder + 'positions/EGF/'
brightfields = BaseFolder + "brightfields/EGF/"

class DataLoader:
    def __init__(self, image_folder, csv_folder, clustering, fourier):
        self.image_folder = image_folder
        self.csv_folder = csv_folder
        self.clustering_transformer = clustering  
        self.fourier_transformer = fourier  
        
        # Get the sorted list of file names to ensure pairing
        # Get the sorted list of image files based on their position
        self.image_files = sorted(
            [f for f in os.listdir(self.image_folder) if f.endswith(('png', 'jpg', 'jpeg', 'tif', 'tiff'))],
            key=self.extract_position_brightfields
        )
        
        # Get the sorted list of CSV files based on their position
        self.csv_files = sorted(
            [f for f in os.listdir(self.csv_folder) if f.endswith('csv')],
            key=self.extract_position_positions
        )

        assert len(self.image_files) == len(self.csv_files), "The number of images and CSV files must be the same."
    
    def __fit__(self):
        processed_data = []
        
        for img_name, csv_name in tqdm(zip(self.image_files, self.csv_files), desc="Processing images and CSVs", total=len(self.image_files)):
            # Full paths to image and CSV
            img_path = os.path.join(self.image_folder, img_name)
            csv_path = os.path.join(self.csv_folder, csv_name)

            processed_image= crop_png(img_path)
            X = trpos_conversion(csv_path)
            image = np.stack((np.array(processed_image),)*3, axis=-1)

             # Apply the clusering DBSCAN on the given position to create the initial mask
            clustered_positions = self.clustering_transformer.__set_dataset__(X).__fit__()
            mask = clustered_positions.__create_mask__()

            # get the brightfield image and add its 
            processed_image = self.fourier_transformer.__set__(image).__fit__()
            
            # Store the processed results
            #processed_data.append((processed_image, clustered_positions))
        
        return self

    def extract_position_brightfields(self,filename):
        parts = filename.split('_')
        for part in parts:
            if part.startswith('posXY'):
                # Extract the number after 'position_' and return it as an integer for sorting
                return int(part.replace('posXY', ''))
        return float('inf')  # Return a large number if 'position' not found to sort it last (though ideally, all files should have it)
    
    def extract_position_positions(self,filename):
        parts = filename.split('_')
        for part in parts:
            if part.startswith('Steps'):
                # Extract the number after 'position_' and return it as an integer for sorting
                return int(part.replace('Steps', ''))
        return float('inf')  # Return a large number if 'p
            

In [26]:
# Initialize objects of type A and B
Clustering_Transformer = ClusteringAlgorithm()
Fourier_Transformer = FourierTransformer()

# Instantiate DataLoader
data_loader = DataLoader(brightfields, positions, Clustering_Transformer, Fourier_Transformer)

# Fit the data loader
processed_data = data_loader.__fit__()

ValueError: invalid literal for int() with base 10: ''